In [21]:
import os
import csv
import pandas as pd
import numpy as np
from IPython.display import clear_output
import ast
import pymongo as pm
from datetime import datetime


### Load connection to database

In [22]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the stanford-cogsci.org user
pswd = auth.values[0][0]
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')

In [23]:
db = conn['devphotodraw_recognition']
coll = db['batched_12afc']

In [24]:
# db = conn['kiddraw']
# coll = db['check_invalid_v5_dev']
# coll.count()

In [25]:
analysis_dir = os.getcwd()
output_dir = os.path.join(analysis_dir,'csv_out')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

### Only prolific workers have id's that are strings (parsed from url)

In [37]:
# prolific_workers = coll.find({'sub_id': {'$type' : "string"}, 'version': 'batch1_production_june2021' }).distinct('sub_id')
prolific_workers = coll.find({'sub_id': {'$type' : "string"}, 'version': 'batch2_production_june2021' }).distinct('sub_id')

np.size(prolific_workers)
prolific_workers

[u'60b3e20053fcb3b9523269f',
 u'5e765cac6382401cbba8a60',
 u'60c7a886807684c1543295d',
 u'5e8cf2990fa7b60ba8b4a65',
 u'60c5a4a48e9b86990001548',
 u'60c7b596fac554f65c66acb',
 u'60c725d89f51b8cbff252bb',
 u'60c3be9465aa1b3e68153df',
 u'60c76eb28fe75982f7c9145',
 u'60c5af39a32d3711a04c7b8']

In [38]:
###### Open up variables for CSV writing
# basic descriptors
subID = []; 
imageName = []
guessed_category = []
drawn_category = [] 

In [39]:
for w in prolific_workers:
    print w
    entries = coll.find({'$and': [{'sub_id': w, 'dataType': 'recognition_rating'}]})
    num_trials =  entries.count()
    print num_trials
    for e in entries:
        subID.append(e['sub_id'])
        imageName.append(e['imageName'])
        guessed_category.append(e['guessed_category'])
        drawn_category.append(e['imageCategory'])

60b3e20053fcb3b9523269f
48
5e765cac6382401cbba8a60
73
60c7a886807684c1543295d
129
5e8cf2990fa7b60ba8b4a65
45
60c5a4a48e9b86990001548
36
60c7b596fac554f65c66acb
105
60c725d89f51b8cbff252bb
26
60c3be9465aa1b3e68153df
5
60c76eb28fe75982f7c9145
59
60c5af39a32d3711a04c7b8
28


In [34]:
e

{u'_id': ObjectId('60c7b9cfc0059952db3ba038'),
 u'dataType': u'recognition_rating',
 u'date': u'2021-06-14T20:19:27.393Z',
 u'guessed_category': u'rabbit',
 u'imageCategory': u'rabbit',
 u'imageName': u'https://stanford-cogsci.org:8880/object_drawings/P_rabbit_sketch_age8_CDM_photodraw_e21550689187843.png',
 u'sub_id': u'60c6d6b60fc74b211f17ddf',
 u'version': u'batch1_production_june2021'}

In [35]:

## write out csv 
X_out = pd.DataFrame([subID, imageName, guessed_category, drawn_category])
X_out = X_out.transpose()
X_out.columns = ['subID','imageName', 'guessed_category','drawn_category']
X_out.to_csv(os.path.join(output_dir,'Devphotodraw_recognition_pilot.csv'))


In [ ]:
# mturk_id
# assignment_id
# trial_id
# hit_id